In [7]:
!pip install textblob
!python -m textblob.download_corpora

  Using cached textblob-0.19.0-py3-none-any.whl.metadata (4.4 kB)
Using cached textblob-0.19.0-py3-none-any.whl (624 kB)
[nltk_data] Downloading package brown to
[nltk_data]     /Users/mayanksingh/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/mayanksingh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mayanksingh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/mayanksingh/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/mayanksingh/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/mayanksingh/nltk_data...
[nltk_data]   Unzipping corpora/movie_rev

In [3]:
from textblob import TextBlob
from langdetect import detect
from googletrans import Translator
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import joblib

In [4]:
df = pd.read_csv("medquad_qa.csv")
vectorizer = joblib.load("saved_model/vectorizer.pkl")
X = joblib.load("saved_model/X.pkl")
translator = Translator()

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
def detect_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity

    if polarity > 0.2:
        return "positive"
    elif polarity < -0.2:
        return "negative"
    else:
        return "neutral"

In [9]:
def sentiment_chatbot(user_input):
    try:
        lang = detect(user_input)
        translated_input = user_input

        # Translate if not English
        if lang != 'en':
            translated_input = translator.translate(user_input, src=lang, dest='en').text

        # Detect sentiment
        sentiment = detect_sentiment(translated_input)

        # TF-IDF response
        user_vec = vectorizer.transform([translated_input])
        similarity = cosine_similarity(user_vec, X).flatten()
        best_idx = similarity.argmax()

        if similarity[best_idx] < 0.2:
            return "I'm sorry, I couldn't find a good answer for that.", lang

        best_answer = df.iloc[best_idx]["answer"]

        # Translate back
        if lang != 'en':
            translated_answer = translator.translate(best_answer, src='en', dest=lang).text
        else:
            translated_answer = best_answer

        # Add sentiment-specific prefix
        if sentiment == "negative":
            prefix = "I'm sorry to hear that. "
        elif sentiment == "positive":
            prefix = "Glad to help! 😊 "
        else:
            prefix = ""

        return prefix + translated_answer, lang

    except Exception as e:
        return f"❌ Error: {e}", "unknown"

In [ ]:
while True:
    user = input("You: ")
    if user.lower() in ['exit', 'quit']:
        print("Bot: Thank you! 🙏")
        break

    reply, user_lang = sentiment_chatbot(user)
    print(f"Bot ({user_lang.upper()}): {reply}\n")

You:  I’m feeling anxious about my chest pain.


Bot (EN): I'm sorry to hear that. Chest pain can be caused by many conditions. If you're feeling anxious too, it's best to consult a doctor immediately.



You:  What should I do for chest pain?


Bot (EN): Chest pain may be serious. Seek medical help if it persists or worsens.

